<a href="https://colab.research.google.com/github/Arunsoren/Pytorch-Basics/blob/main/Neural_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image                        # for images
import torchvision.transforms as transforms
import torchvision.models as models          #for VGG
from torchvision.utils import save_image


In [2]:
model = models.vgg19(pretrained=True).features
# 0-4, 5-9, 10-18, 19-27, 28-  #conv blocks
#model = MyModel()

if torch.cuda.is_available():
    model.cuda()


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [3]:
print(model)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [4]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()

        self.chosen_features = ['0','5', '10','19', '28']
        self.model = models.vgg19(pretrained=True).features[:29]

    def forward(self, x):
        features = []

        for layers_num, layer in enumerate(self.model):
            x = layer(x)

            if str(layers_num) in self.chosen_features:
                features.append(x)

        return features

In [5]:
def load_image(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
image_size = 356

In [7]:
print(device)

cuda


In [8]:

loader = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[], std=[])
    ]
)


In [9]:
original_img = load_image("/content/Original.png")
style_img = load_image("/content/style.png")

In [10]:
generated = original_img.clone().requires_grad_(True).to(device)

In [11]:
# Hyperparameters

total_steps  = 6000
learning_rate = 0.001
alpha = 1.0
beta = 0.01
optimizer = optim.Adam([generated], lr=learning_rate)

In [12]:
for step in range(total_steps):
    print("STEP:", step)  
    # Obtain the convolution features in specifically chosen layers
    generated_features = model(generated)
    #print("generated_features:",generated_features.shape)
    original_img_features = model(original_img)
    #print("original_img_features:", original_img_features.shape)
    style_features = model(style_img)
    #print("style_features:", style_features.shape)

    # Loss is 0 initially
    style_loss = original_loss = 0
    batch_size =1

    # iterate through all the features for the chosen layers
    for gen_feature, orig_feature, style_feature in zip(
        generated_features, original_img_features, style_features
    ):

        # batch_size will just be 1
        print("gen_feature.shape:", gen_feature.shape)
        channel, height, width = gen_feature.shape
        original_loss += torch.mean((gen_feature - orig_feature) ** 2)
        # Compute Gram Matrix of generated
        G = gen_feature.view(channel, height * width).mm(
            gen_feature.view(channel, height * width).t()
        )
        # Compute Gram Matrix of Style
        A = style_feature.view(channel, height * width).mm(
            style_feature.view(channel, height * width).t()
        )
        style_loss += torch.mean((G - A) ** 2)

    total_loss = alpha * original_loss + beta * style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if step % 200 == 0:
        print(total_loss)
        save_image(generated, "generated.png")

Streaming output truncated to the last 5000 lines.
STEP: 3506
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3507
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3508
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3509
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3510
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3511
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3512
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3513
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3514
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3515
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3516
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3517
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3518
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3519
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3520
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3521
gen_feature.shape: torch.Size([512, 11, 11])
STEP: 3522
gen_feature.shape: torch.Size([512, 11, 11